In [244]:
import os
import re
import json
import random
import sys
import pickle
import datetime
import asyncio
import nest_asyncio
import sympy as sp
import gurobipy as gp

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from colorama import Fore, Style
from pydantic import BaseModel
from typing import List
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.llms.lmstudio import LMStudio

sys.path.append('../')
from utils import *

nest_asyncio.apply()

In [4]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'LPWP.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-18 20:33:30.899 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\LPWP.txt
2024-09-18 20:33:30.900 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\LPWP.txt
2024-09-18 20:33:30.901 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-18 20:33:30.902 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [5]:
llm = LMStudio(
    model_name="bartowski/DeepSeek-Coder-V2-Lite-Instruct-GGUF",
    base_url="http://localhost:1234/v1",
    temperature=0.0,
)

In [12]:
class Code(BaseModel):
    # reasoning: List[str]
    code: str     

In [127]:
prompt_template_str = """You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into a piece of code.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 13 * y <= 1345)
m.addConstr(43 * x + 4 * y <= 346)
m.addConstr(56 * x + 45 * y <= 1643)

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
CODE:
x = m.addVar(name="apples", vtype=gp.GRB.INTEGER)
y = m.addVar(name="pears", vtype=gp.GRB.INTEGER)
m.setObjective(2 * x + 4 * y, gp.GRB.MAXIMIZE)
m.addConstr(x + y <= 50)
m.addConstr(x >= 5)
m.addConstr(y >= 10)
m.addConstr(y <= 2 * x)

Follow the above steps and try to step by step generate the code for the following question.
"""

prompt_template_str = prompt_template_str + example + "\nQUESTIOM: {q}"
print(prompt_template_str)

You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into a piece of code.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEG

In [128]:
program = LLMTextCompletionProgram.from_defaults(
    output_cls=Code,
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=False,
)

In [129]:
def parse_value_error_message(text: str) -> str:
    # Regular expression to capture everything after "CODE:"
    pattern = r'CODE:\s*(.*)'

    # Extracting the code after "CODE:"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        code = match.group(1).strip()  # Extract the captured group and strip extra spaces
        return code
    else:
        return None

In [144]:
codes = {}
for i in tqdm(range(len(questions))):
    try:
        codes[i] = program(q=questions[i])
    except Exception as e:
        # loguru.logger.error(f"Error for question {i}: {e}")
        codes[i] = f"{e}"
        continue

100%|██████████| 288/288 [36:25<00:00,  7.59s/it]


In [145]:
while True:
    error_idx = []
    for k, v in codes.items():
        if isinstance(v, str):
            if v.startswith("Could not extract json string from output:"):
                codes[k] = parse_value_error_message(v)
            elif v.startswith("1 validation error for Code"):
                error_idx.append(k)
    
    print(error_idx)
    if len(error_idx) == 0:
        break

    for ei in tqdm(error_idx):
        try:
            codes[ei] = program(q=questions[ei])
        except Exception as e:
            # loguru.logger.error(f"Error for question {i}: {e}")
            codes[ei] = f"{e}"
            continue

loguru.logger.info("All questions have been answered")

[1, 5, 7, 10, 34, 48, 54, 87, 93, 99, 106, 116, 124, 132, 190, 195, 198, 206, 214, 219, 224, 231, 236, 239, 241, 271, 280]


100%|██████████| 27/27 [03:17<00:00,  7.30s/it]


[132, 214, 271]


100%|██████████| 3/3 [00:21<00:00,  7.20s/it]
2024-09-19 00:52:01.416 | INFO     | __main__:<module>:22 - All questions have been answered


[]


In [146]:
code_strs = [codes[i].code if isinstance(codes[i], Code) else codes[i] for i in range(len(codes)) ]

In [150]:
len(code_strs)

288

In [154]:
filename = 'e2e_codegen_deepseek_coderv2_lpwp_gurobi_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(codes, f)

In [155]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [294]:
def clean_code(code: str) -> str:
    temp_code = code
    # Split the code into lines
    pattern = r'\)([a-zA-Z])'
    temp_code = re.sub(pattern, r')\n\1', temp_code)

    cleand_code = []
    for line in temp_code.split('\n'):
        line = line.strip()
        # Replace > < to >= <=
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            # print("Not found")
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        # Remove all comments and suffix and prefix terms
        if not line.startswith('```') and not line.startswith('#'):
            cleand_code.append(line)
        else:
            continue
        # Don't support bool expression '=='
        if re.findall(r'==', line):
            cleand_code.remove(line)
        
    cleand_code = '\n'.join(cleand_code)

    # Remove all '{' and '}'
    cleand_code = cleand_code.replace('{', '').replace('}', '')
    return cleand_code

In [295]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [296]:
def get_variables(code: List[str]) -> List[str]:
    vars = []
    for line in code:
        if re.findall(r'addVar', line):
            pattern = r'(\w+)\s*='
            matches = re.findall(pattern, line)
            vars.append(matches[0])
    return sp.symbols(vars)

In [354]:
def simplify_code(code: str) -> str:
    simplfied_code = []
    for i, line in enumerate(code.split('\n')):
        if line.startswith('m.addConstr') or line.startswith('m.setObjective'):
            if '/' in line:
                obj_pattern = r'm\.setObjective\(([^,]*)'
                constr_pattern = r'm\.addConstr\((.*)\)'
                if re.findall(obj_pattern, line):
                    matches = re.findall(obj_pattern, line)
                    obj = re.search(r'gp\.GRB\.(\w+)', line).group(1)
                    expr = sp.sympify(matches[0])
                    simplfied_code.append(f"m.setObjective({str(sp.simplify(expr))}, gp.GRB.{obj})")
                if re.findall(constr_pattern, line):
                    matches = re.findall(constr_pattern, line)
                    oper = re.search(r'\s*(>=|<=)\s*', matches[0]).group(1)
                    expr = sp.sympify(matches[0])
                    simplified_expr = str(sp.simplify(expr.lhs - expr.rhs))
                    if match := re.search(r'^\((.*?)\)/', simplified_expr):
                        new_constr = f'{match.group(1)} {oper} {str(0)}'
                        simplfied_code.append('m.addConstr(' + new_constr + ')')
            else:
                simplfied_code.append(line)
        else:
            simplfied_code.append(line)
    return '\n'.join(simplfied_code)

In [356]:
pred_answers = []
for i, code_str in enumerate(code_strs):
    try:
        cl_code = clean_code(code_str)
        si_code = simplify_code(cl_code)
        co_code = complement_code(si_code)
        ans = execute_code(co_code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-19 12:56:56.094 | ERROR    | __main__:<module>:11 - Error for question 0: name 'budget' is not defined
2024-09-19 12:56:56.096 | INFO     | __main__:<module>:8 - question 1 obtain answer
2024-09-19 12:56:56.099 | INFO     | __main__:<module>:8 - question 2 obtain answer
2024-09-19 12:56:56.100 | INFO     | __main__:<module>:8 - question 3 obtain answer
2024-09-19 12:56:56.102 | INFO     | __main__:<module>:8 - question 4 obtain answer
2024-09-19 12:56:56.104 | INFO     | __main__:<module>:8 - question 5 obtain answer
2024-09-19 12:56:56.105 | INFO     | __main__:<module>:8 - question 6 obtain answer
2024-09-19 12:56:56.106 | INFO     | __main__:<module>:8 - question 7 obtain answer
2024-09-19 12:56:56.108 | INFO     | __main__:<module>:8 - question 8 obtain answer
2024-09-19 12:56:56.110 | INFO     | __main__:<module>:8 - question 9 obtain answer
2024-09-19 12:56:56.112 | INFO     | __main__:<module>:8 - question 10 obtain answer
2024-09-19 12:56:56.114 | INFO     | __main__:<m

Not found
Not found
Not found
Not found
Not found
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 3 nonzeros
Model fingerprint: 0x5ffc7e62
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1000 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+03, best bound 1.000000000000e+03, gap 0.0000%


2024-09-19 12:56:56.199 | INFO     | __main__:<module>:8 - question 51 obtain answer
2024-09-19 12:56:56.217 | ERROR    | __main__:<module>:11 - Error for question 52: name 'y_process2' is not defined
2024-09-19 12:56:56.220 | INFO     | __main__:<module>:8 - question 53 obtain answer
2024-09-19 12:56:56.222 | INFO     | __main__:<module>:8 - question 54 obtain answer


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xe6cd2430
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 515.0000000
Found heuristic solution: objective 516.0000000

Root relaxation: objective 5.710000e+02, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

2024-09-19 12:56:56.233 | INFO     | __main__:<module>:8 - question 55 obtain answer
2024-09-19 12:56:56.235 | INFO     | __main__:<module>:8 - question 56 obtain answer


...

Not found


2024-09-19 12:56:56.993 | INFO     | __main__:<module>:8 - question 276 obtain answer
2024-09-19 12:56:57.001 | ERROR    | __main__:<module>:11 - Error for question 277: Constraint has no bool value (are you trying "lb <= expr <= ub"?)
2024-09-19 12:56:57.003 | INFO     | __main__:<module>:8 - question 278 obtain answer
2024-09-19 12:56:57.005 | INFO     | __main__:<module>:8 - question 279 obtain answer
2024-09-19 12:56:57.007 | INFO     | __main__:<module>:8 - question 280 obtain answer


Not found


2024-09-19 12:56:57.008 | INFO     | __main__:<module>:8 - question 281 obtain answer
2024-09-19 12:56:57.010 | INFO     | __main__:<module>:8 - question 282 obtain answer
2024-09-19 12:56:57.012 | INFO     | __main__:<module>:8 - question 283 obtain answer
2024-09-19 12:56:57.013 | INFO     | __main__:<module>:8 - question 284 obtain answer
2024-09-19 12:56:57.015 | INFO     | __main__:<module>:8 - question 285 obtain answer
2024-09-19 12:56:57.016 | INFO     | __main__:<module>:8 - question 286 obtain answer
2024-09-19 12:56:57.017 | INFO     | __main__:<module>:8 - question 287 obtain answer


Not found


In [357]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [358]:
print(f"Accuracy under error {1e-2}: {sum(mark(pred_answers, answers[:35], 1e-2)) / len(answers[:35]) * 100}")
print(f"Accuracy under error {1e-4}: {sum(mark(pred_answers, answers[:35], 1e-4)) / len(answers[:35]) * 100}")

Accuracy under error 0.01: 57.14285714285714
Accuracy under error 0.0001: 54.285714285714285
